In [ ]:
import duckdb

In [ ]:
duckdb.sql("CREATE TABLE bromley AS SELECT * FROM 'data/43967__Max_Min_Temperature__H.csv'")

In [ ]:
duckdb.sql("""DESCRIBE bromley""")

In [ ]:
duckdb.sql("""
           SELECT "Observation time UTC"::DATE AS day
               , MIN("Minimum Temperature [Deg C]") AS sparrows_min
               , MAX("Maximum Temperature [Deg C]") AS sparrows_max
               , ROUND(AVG("Mean Temperature [Deg C]"), 1) AS sparrows_average
           FROM bromley 
           WHERE strftime('%H', "Observation time UTC") BETWEEN '06' AND '09'
           GROUP BY "Observation time UTC"::DATE
           HAVING COUNT(*) = 4
           """)


In [ ]:
duckdb.sql("""
            WITH local_time AS (
                SELECT timezone('Pacific/Auckland', timezone('UTC', "Observation time UTC")) AS time
                        , strftime('%m', timezone('Pacific/Auckland', timezone('UTC', "Observation time UTC"))) AS month
                        , "Minimum Temperature [Deg C]" AS sparrows_min 
                        , "Maximum Temperature [Deg C]" AS sparrows_max
                        , "Mean Temperature [Deg C]" AS sparrows_average
                FROM bromley            
            )
            SELECT strftime('%B', ('2000-' || month || '-01')::Date) as month_name
                , MIN(sparrows_min) as min_temp 
                , ROUND(AVG(sparrows_min), 1) as mean_min_temp 
                , MAX(sparrows_max) as max_temp 
                , ROUND(AVG(sparrows_max), 1) as mean_max_temp 
                , ROUND(AVG(sparrows_average), 1) as mean_temp 
                , ROUND(stddev_samp(sparrows_average), 1) as stdev_temp
            FROM local_time
            WHERE strftime('%H', time) BETWEEN '05' AND '09'
            GROUP BY month
            ORDER BY month
            """)
